# Exploración de los datos. 

Los datos que analizamos vienen de la [página de la Comisión de Taxis y Limosinas de NYC](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). Nosotros analizamos la información de los taxis verdes de NYC (hay verdes y amarillos) para el año 2019, dividiéndonos el trabajo por cuatrimestre. En este notebook exploraré un poco los datos y empezaré a crear un pipeline para preparar la información de cada mes y obtener estadísticas relevantes. 

Cabe mencionar que subimos al repo el diccionario de datos que la página oficial provee. De ahí obtuvimos la interpretación de varias variables. 

# 1- Limpieza de Datos 

#### Cargandos los datos

Empezaré por el mes de septiembre, el primer mes del último cuatrimestre (el que me tocó explorar)

In [8]:
sept <- read.csv('septiembre.csv')
head(sept)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,2,2019-09-01 00:10:53,2019-09-01 00:23:46,N,1,65,189,5,2.00,10.5,0.5,0.5,2.36,0.00,NA,0.3,14.16,1,1,0
2,2,2019-09-01 00:31:22,2019-09-01 00:44:37,N,1,97,225,5,3.20,12.0,0.5,0.5,0.00,0.00,NA,0.3,13.30,2,1,0
3,2,2019-09-01 00:50:24,2019-09-01 01:03:20,N,1,37,61,5,2.99,12.0,0.5,0.5,0.00,0.00,NA,0.3,13.30,2,1,0
4,2,2019-09-01 00:27:06,2019-09-01 00:33:22,N,1,145,112,1,1.73,7.5,0.5,0.5,1.50,0.00,NA,0.3,10.30,1,1,0
5,2,2019-09-01 00:43:23,2019-09-01 00:59:54,N,1,112,198,1,3.42,14.0,0.5,0.5,3.06,0.00,NA,0.3,18.36,1,1,0
6,2,2019-09-01 00:55:37,2019-09-01 01:17:02,N,1,7,41,1,6.26,21.0,0.5,0.5,0.00,6.12,NA,0.3,28.42,2,1,0


In [9]:
#veamos el número de columnas y de renglones 
dim(sept)

[1] 449063     20

Casi medio millón de registros! En verdad, no analizamos la información de los taxis amarillos porque cada mes tenía más de 6 millones de registros. 

### Explorando los valores nulos en los datos

In [10]:
library(tidyverse)
sept %>% summarise_all(~ sum(is.na(.)))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
59143,0,0,0,59143,0,0,59143,0,0,0,0,0,0,449063,0,0,59143,59143,59143


Antes de proceder a explicar el procedimiento respecto de los datos nulos, podemos tirar las columnas que no nos sirven por su poco poder explicativo en el análisis que queremos hacer. 

#### Tirar columnas que no son de interés. 

Existen varias columnas que indican códigos que no necesariamente harán más profundo el análisis estadístico, por lo que podemos prescindir de ellas. 

Entre ellas se encuentran las siguientes, pero hay más:

VendorID

Store_and_fwd_flag

PULocationID

DOLocationID

RateCodeID


(Se puede consultar el diccionario de datos en la página citada para descubrir por qué)

In [178]:
#creemos una función que nos ahorra el trabajo futuro 

delete_cols <- function(data_frame) {
    #function to remove unnecessary cols in r dataframe
    data_frame$VendorID <- NULL
    data_frame$store_and_fwd_flag <- NULL
    data_frame$PULocationID <- NULL
    data_frame$DOLocationID <- NULL
    data_frame$RatecodeID <- NULL
    data_frame$ehail_fee <- NULL
    return(data_frame)
}

In [179]:
clean_data <- delete_cols(sept)

In [180]:
#tiempo de ver los valores vacíos de nuevo
colSums(is.na(clean_data))

lpep_pickup_datetime lpep_dropoff_datetime       passenger_count 
                    0                     0                 59143 
        trip_distance           fare_amount                 extra 
                    0                     0                     0 
              mta_tax            tip_amount          tolls_amount 
                    0                     0                     0 
improvement_surcharge          total_amount          payment_type 
                    0                     0                 59143 
            trip_type  congestion_surcharge 
                59143                 59143

In [181]:
dim(clean_data)

[1] 449063     14

Respecto a los valores nulos, por simplicidad podemos tirarlos. No representan una gran proporción de los datos y será más fácil después realizar el análisis de regresión lineal sin datos nulos. Además, no hace sentido intentar extrapolar, por ejemplo, cuántos pasajeros había en  los 59 mil viajes donde no se registraron. Dado que este es un ejercicio de estadística descriptiva, lo mejor es tirarlos para que no introduzcan ruido. Además, como veremos a continuación, los datos nulos estaban en los mismos renglones porque el dataframe que no los incluye tiene el número de renglones original menos el número de renglones con nulos de algunas variables. 

In [182]:
head(clean_data[complete.cases(clean_data),])

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
1,2019-09-01 00:10:53,2019-09-01 00:23:46,5,2.00,10.5,0.5,0.5,2.36,0.00,0.3,14.16,1,1,0
2,2019-09-01 00:31:22,2019-09-01 00:44:37,5,3.20,12.0,0.5,0.5,0.00,0.00,0.3,13.30,2,1,0
3,2019-09-01 00:50:24,2019-09-01 01:03:20,5,2.99,12.0,0.5,0.5,0.00,0.00,0.3,13.30,2,1,0
4,2019-09-01 00:27:06,2019-09-01 00:33:22,1,1.73,7.5,0.5,0.5,1.50,0.00,0.3,10.30,1,1,0
5,2019-09-01 00:43:23,2019-09-01 00:59:54,1,3.42,14.0,0.5,0.5,3.06,0.00,0.3,18.36,1,1,0
6,2019-09-01 00:55:37,2019-09-01 01:17:02,1,6.26,21.0,0.5,0.5,0.00,6.12,0.3,28.42,2,1,0


In [183]:
dim((clean_data[complete.cases(clean_data),]))

[1] 389920     14

In [184]:
449063 - 389920  #filas antes - filas sin nulos

[1] 59143

#### Podemos ir creando una función de limpieza más completa entonces  

In [185]:
clean_dataframe <- function(data_frame) {
    #function to remove unnecessary cols in r dataframe and na's
    data_frame$VendorID <- NULL
    data_frame$store_and_fwd_flag <- NULL
    data_frame$PULocationID <- NULL
    data_frame$DOLocationID <- NULL
    data_frame$RatecodeID <- NULL
    data_frame$ehail_fee <- NULL
    data_frame$improvement_surcharge <- NULL 
    data_frame$tolls_amount <- NULL 
    data_frame$mta_tax <- NULL 
    data_frame$extra <- NULL 
    data_frame$congestion_surcharge <- NULL
    return(data_frame[complete.cases(data_frame),])
}

In [186]:
clean_data <- clean_dataframe(sept)

In [187]:
head(clean_data)

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,2019-09-01 00:10:53,2019-09-01 00:23:46,5,2.00,10.5,2.36,14.16,1,1
2,2019-09-01 00:31:22,2019-09-01 00:44:37,5,3.20,12.0,0.00,13.30,2,1
3,2019-09-01 00:50:24,2019-09-01 01:03:20,5,2.99,12.0,0.00,13.30,2,1
4,2019-09-01 00:27:06,2019-09-01 00:33:22,1,1.73,7.5,1.50,10.30,1,1
5,2019-09-01 00:43:23,2019-09-01 00:59:54,1,3.42,14.0,3.06,18.36,1,1
6,2019-09-01 00:55:37,2019-09-01 01:17:02,1,6.26,21.0,0.00,28.42,2,1


In [188]:
dim(clean_data)

[1] 389920      9

# Encontrando la duración de los viajes

Quiero encontrar la forma de restar dropoff menos pickup para encontrar la duración de un viaje...

In [189]:
copy <- clean_data[,]

In [190]:
typeof(copy$lpep_pickup_datetime)

[1] "character"

In [191]:
copy$lpep_dropoff_datetime <- as.POSIXct(copy$lpep_dropoff_datetime,
                                      format='%Y-%m-%d %H:%M:%S')
copy$lpep_pickup_datetime <- as.POSIXct(copy$lpep_pickup_datetime,
                                      format='%Y-%m-%d %H:%M:%S')

In [192]:
head(copy)

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,2019-09-01 00:10:53,2019-09-01 00:23:46,5,2.00,10.5,2.36,14.16,1,1
2,2019-09-01 00:31:22,2019-09-01 00:44:37,5,3.20,12.0,0.00,13.30,2,1
3,2019-09-01 00:50:24,2019-09-01 01:03:20,5,2.99,12.0,0.00,13.30,2,1
4,2019-09-01 00:27:06,2019-09-01 00:33:22,1,1.73,7.5,1.50,10.30,1,1
5,2019-09-01 00:43:23,2019-09-01 00:59:54,1,3.42,14.0,3.06,18.36,1,1
6,2019-09-01 00:55:37,2019-09-01 01:17:02,1,6.26,21.0,0.00,28.42,2,1


In [193]:
#una vez que tenemos estas columnas en formato datetime, podemos calcular la diferencia

In [194]:
copy$trip_duration <- as.numeric(difftime(copy$lpep_dropoff_datetime, copy$lpep_pickup_datetime, units="hours"))

# Algo misterioso ocurrió. 

¿En verdad ha habido viajes de 23 horas? ¿O fue un error en la medición de los datos? ¿Es posible que te cobren 12 o 15 dólares por un viaje de 23 horas, como se muestra a continuación?

In [195]:
suspect <- copy[copy$trip_duration>5,]
mean(suspect$trip_duration) #el promedio de la duración de los viajes del dataframe suspect es de 21 horas!

[1] 21.14084

In [196]:
head(suspect)

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
41,2019-08-31 23:12:04,2019-09-01 22:30:13,1,2.57,13.5,0.00,14.80,2,1,23.30250
244,2019-09-01 00:02:39,2019-09-01 23:41:29,1,4.77,18.5,0.00,19.80,1,1,23.64722
270,2019-09-01 00:00:05,2019-09-01 23:18:29,1,2.11,11.0,0.00,12.30,2,1,23.30667
405,2019-09-01 00:27:04,2019-09-02 00:12:56,1,0.89,5.0,1.26,7.56,1,1,23.76444
443,2019-08-31 22:07:50,2019-09-01 21:29:08,1,5.09,20.0,4.26,25.56,1,1,23.35500
637,2019-08-31 23:07:11,2019-09-01 22:22:13,1,3.04,13.0,1.80,16.10,1,1,23.25056


In [197]:
dim(suspect)

[1] 1973   10

In [198]:
mean(suspect$total_amount) #pero solo te cobran 25 dólares? Por 20 horas de viaje? 

[1] 24.97581

Claro, y aquí vemos que en realidad estos datos MUY probablemente estén mal. Un promedio de 25 dólares de cobro por 21 horas de viaje? Cuando los datos originales tienen un costo mucho mayor. 

In [199]:
normal <- copy[copy$trip_duration<5,]
mean(normal$total_amount)

[1] 15.8603

In [200]:
mean(normal$trip_duration)

[1] 0.2381359

Vemos un costo promedio de casi 16 dólares por viajes que en promedio duran una cuarto de hora, es decir, 15 minutos. Podemos tirar entonces los datos con duración de viaje de más de 5 horas. 

### Cobros negativos?

Otra cosa que llama la atención en el dataframe es que hay renglones para los cuales total_amount, tip_amount y fare_amount es negativo, lo cual no hace sentido. Eliminémoslos. 

Nota: No se hace mención de esto en el diccionario de datos.

In [201]:
head(suspect[suspect$total_amount < 0, ])

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
94095,2019-09-08 14:06:42,2019-09-09 13:12:10,2,0.24,-3.0,0,-3.8,4,1,23.09111
225803,2019-09-18 16:34:12,2019-09-19 16:33:01,1,0.00,-3.0,0,-4.8,3,1,23.98028
344322,2019-09-27 11:49:34,2019-09-28 11:10:38,2,0.01,-2.5,0,-3.3,3,1,23.35111
387076,2019-09-30 18:09:20,2019-10-01 18:09:10,3,0.31,-3.5,0,-5.3,3,1,23.99722


In [203]:
head(suspect[suspect$fare_amount < 0, ])

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
94095,2019-09-08 14:06:42,2019-09-09 13:12:10,2,0.24,-3.0,0,-3.8,4,1,23.09111
225803,2019-09-18 16:34:12,2019-09-19 16:33:01,1,0.00,-3.0,0,-4.8,3,1,23.98028
344322,2019-09-27 11:49:34,2019-09-28 11:10:38,2,0.01,-2.5,0,-3.3,3,1,23.35111
387076,2019-09-30 18:09:20,2019-10-01 18:09:10,3,0.31,-3.5,0,-5.3,3,1,23.99722


In [204]:
clean_data[clean_data$tip_amount <0,]

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
14946,2019-09-02 14:25:36,2019-09-02 14:26:49,1,0,-3.00,-1.14,-4.94,3,1
117805,2019-09-10 13:56:43,2019-09-10 13:57:34,1,0,-2.50,-0.99,-4.29,3,1
125385,2019-09-10 23:36:44,2019-09-10 23:36:51,1,0,-15.55,-3.89,-19.44,4,2
235948,2019-09-19 12:50:50,2019-09-19 12:52:04,1,0,-3.00,-0.76,-4.56,3,1
236682,2019-09-19 13:22:01,2019-09-19 13:22:46,1,0,-2.50,-0.66,-3.96,3,1
248773,2019-09-20 10:19:45,2019-09-20 10:21:13,1,0,-3.00,-1.14,-4.94,3,1
290642,2019-09-23 10:24:17,2019-09-23 10:30:07,1,0,-3.00,-0.76,-4.56,3,1
332570,2019-09-26 15:06:45,2019-09-26 15:07:24,1,0,-2.50,-0.99,-4.29,3,1
373239,2019-09-29 13:48:42,2019-09-29 13:48:42,1,0,-2.50,-0.66,-3.96,3,1


In [209]:
summary(clean_data$payment_type)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   1.000   1.456   2.000   5.000 

### Hay columnas que es necesario crear por nuestra cuenta. 

Además de la columna que calcular el tiempo del viaje, sería útil tener una columna que señale el porcentaje que representa la propina del costo total.

In [142]:
#probando lo de la propina 
copy$tip_pct <- copy$tip_amount / copy$total_amount
head(copy)

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration,tip_pct
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>
1,2019-09-01 00:10:53,2019-09-01 00:23:46,5,2.00,10.5,2.36,14.16,1,1,0.2147222,0.1666667
2,2019-09-01 00:31:22,2019-09-01 00:44:37,5,3.20,12.0,0.00,13.30,2,1,0.2208333,0.0000000
3,2019-09-01 00:50:24,2019-09-01 01:03:20,5,2.99,12.0,0.00,13.30,2,1,0.2155556,0.0000000
4,2019-09-01 00:27:06,2019-09-01 00:33:22,1,1.73,7.5,1.50,10.30,1,1,0.1044444,0.1456311
5,2019-09-01 00:43:23,2019-09-01 00:59:54,1,3.42,14.0,3.06,18.36,1,1,0.2752778,0.1666667
6,2019-09-01 00:55:37,2019-09-01 01:17:02,1,6.26,21.0,0.00,28.42,2,1,0.3569444,0.0000000


In [1]:
#Funciones relevantes hasta ahora:

clean_dataframe <- function(data_frame) {
    #function to remove unnecessary cols in r dataframe and na's
    data_frame$VendorID <- NULL
    data_frame$store_and_fwd_flag <- NULL
    data_frame$PULocationID <- NULL
    data_frame$DOLocationID <- NULL
    data_frame$RatecodeID <- NULL
    data_frame$ehail_fee <- NULL
    data_frame$improvement_surcharge <- NULL 
    data_frame$tolls_amount <- NULL 
    data_frame$mta_tax <- NULL 
    data_frame$extra <- NULL 
    data_frame$congestion_surcharge <- NULL
    return(data_frame[complete.cases(data_frame),])
}

add_columns <- function(data_frame, month) {
    #function to add 2 columns to data frame to improve analysis
    #first we add trip duration in hours
    data_frame$lpep_dropoff_datetime <- as.POSIXct(data_frame$lpep_dropoff_datetime,
                                      format='%Y-%m-%d %H:%M:%S')
    data_frame$lpep_pickup_datetime <- as.POSIXct(data_frame$lpep_pickup_datetime,
                                      format='%Y-%m-%d %H:%M:%S')
    data_frame$trip_duration <- as.numeric(difftime(data_frame[,2], data_frame[,1], units="hours"))
    
    #then we add tip percentage
    data_frame$tip_pct <- data_frame$tip_amount / data_frame$total_amount
    data_frame$month <- month
    data_frame <- data_frame[data_frame$trip_duration<5,]
    data_frame <- data_frame[data_frame$total_amount > 0,]
    data_frame <- data_frame[data_frame$fare_amount > 0,]
    data_frame <- data_frame[data_frame$tip_amount > 0,]
    data_frame <- data_frame[data_frame$trip_duration >0,]
    data_frame <- data_frame[data_frame$trip_distance >0,]
    return(data_frame)
}


data_pipeline <- function(csv, month){
    #function to include all cleaning and transformation functions 
    data <- read.csv(csv)
    data <- clean_dataframe(data)
    data <- add_columns(data, month)
    return(data)
}


In [2]:
test <- data_pipeline('septiembre.csv', 'Septiembre')
head(test)

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration,tip_pct,month
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
1,2019-09-01 00:10:53,2019-09-01 00:23:46,5,2.00,10.5,2.36,14.16,1,1,0.2147222,0.1666667,Septiembre
4,2019-09-01 00:27:06,2019-09-01 00:33:22,1,1.73,7.5,1.50,10.30,1,1,0.1044444,0.1456311,Septiembre
5,2019-09-01 00:43:23,2019-09-01 00:59:54,1,3.42,14.0,3.06,18.36,1,1,0.2752778,0.1666667,Septiembre
7,2019-09-01 00:28:55,2019-09-01 00:52:09,1,4.34,18.0,3.86,23.16,1,1,0.3872222,0.1666667,Septiembre
15,2019-09-01 00:13:37,2019-09-01 00:26:46,1,3.59,13.0,5.12,22.17,1,1,0.2191667,0.2309427,Septiembre
16,2019-09-01 00:35:43,2019-09-01 00:47:21,1,2.10,9.5,1.70,13.00,1,1,0.1938889,0.1307692,Septiembre


In [3]:
#funciona con otros meses también
oct <- data_pipeline('octubre.csv', 'Octubre')
head(oct)

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration,tip_pct,month
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
5,2019-10-01 00:08:13,2019-10-01 00:17:56,1,2.52,10,2.26,13.56,1,1,0.16194444,0.16666667,Octubre
6,2019-10-01 00:35:01,2019-10-01 00:43:40,1,1.47,8,1.86,11.16,1,1,0.14416667,0.16666667,Octubre
7,2019-10-01 00:28:09,2019-10-01 00:30:49,1,0.60,4,1.00,6.30,1,1,0.04444444,0.15873016,Octubre
10,2019-10-01 00:03:03,2019-10-01 00:17:13,1,3.40,13,2.85,17.15,1,1,0.23611111,0.16618076,Octubre
12,2019-10-01 00:25:48,2019-10-01 00:49:52,1,4.70,18,1.00,20.30,1,1,0.40111111,0.04926108,Octubre
14,2019-10-01 00:44:56,2019-10-01 00:51:06,1,1.19,10,0.25,10.25,1,2,0.10277778,0.02439024,Octubre


# 2- Uniendo los distintos meses

Será más fácil trabajar con un dataframe que incluya todos los meses. Hay que concatenar los dataframes. 


In [4]:
nov <- data_pipeline('noviembre.csv', 'Noviembre')
dic <- data_pipeline('diciembre.csv', 'Diciembre')

In [6]:
print("Dimensiones de Septiembre:")
print(dim(test))
print("Dimensiones de Octubre:")
print(dim(oct))
print("Dimensiones de Noviembre:")
print(dim(nov))
print("Dimensiones de Diciembre:")
print(dim(dic))

[1] "Dimensiones de Septiembre:"
[1] 154789     12
[1] "Dimensiones de Octubre:"
[1] 164296     12
[1] "Dimensiones de Noviembre:"
[1] 156204     12
[1] "Dimensiones de Diciembre:"
[1] 151713     12


Lo importante es que tengan las mismas columnas. Tiempo de concatenarlos. 

In [11]:
total_data <- bind_rows(test,oct,nov,dic)
dim(total_data) #la suma de todas

[1] 627002     12

In [214]:
head(total_data) #inicia con septiembre

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration,tip_pct,month
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
1,2019-09-01 00:10:53,2019-09-01 00:23:46,5,2.00,10.5,2.36,14.16,1,1,0.2147222,0.1666667,Septiembre
2,2019-09-01 00:27:06,2019-09-01 00:33:22,1,1.73,7.5,1.50,10.30,1,1,0.1044444,0.1456311,Septiembre
3,2019-09-01 00:43:23,2019-09-01 00:59:54,1,3.42,14.0,3.06,18.36,1,1,0.2752778,0.1666667,Septiembre
4,2019-09-01 00:28:55,2019-09-01 00:52:09,1,4.34,18.0,3.86,23.16,1,1,0.3872222,0.1666667,Septiembre
5,2019-09-01 00:13:37,2019-09-01 00:26:46,1,3.59,13.0,5.12,22.17,1,1,0.2191667,0.2309427,Septiembre
6,2019-09-01 00:35:43,2019-09-01 00:47:21,1,2.10,9.5,1.70,13.00,1,1,0.1938889,0.1307692,Septiembre


In [215]:
tail(total_data) #acaba con diciembre

,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration,tip_pct,month
,<dttm>,<dttm>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
633869,2019-12-31 23:47:50,2020-01-01 00:04:53,1,3.84,14.5,3.71,22.26,1,1,0.28416667,0.1666667,Diciembre
633870,2019-12-31 23:21:37,2019-12-31 23:47:09,1,14.61,40.5,7.19,55.11,1,1,0.42555556,0.1304663,Diciembre
633871,2019-12-31 23:40:39,2019-12-31 23:43:46,1,0.93,4.5,5.00,10.80,1,1,0.05194444,0.4629630,Diciembre
633872,2019-12-31 23:17:45,2019-12-31 23:38:27,1,6.45,22.0,3.50,28.75,1,1,0.34500000,0.1217391,Diciembre
633873,2019-12-31 23:07:37,2019-12-31 23:26:38,1,3.77,15.5,5.04,21.84,1,1,0.31694444,0.2307692,Diciembre
633874,2019-12-31 23:21:06,2019-12-31 23:47:15,1,8.27,27.5,6.31,37.86,1,1,0.43583333,0.1666667,Diciembre


# 3 - Exportar datos 

El análisis estadístico será realizado en otro notebook sobre el csv que exportaremos ahora. 

In [216]:
write.csv(x=total_data, file="cuarto_cuatrimestre.csv")

In [217]:
#asegurémonos de que todo bien

prueba <- read.csv('cuarto_cuatrimestre.csv')
head(prueba)

,X,lpep_pickup_datetime,lpep_dropoff_datetime,passenger_count,trip_distance,fare_amount,tip_amount,total_amount,payment_type,trip_type,trip_duration,tip_pct,month
,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
1,1,2019-09-01 00:10:53,2019-09-01 00:23:46,5,2.00,10.5,2.36,14.16,1,1,0.2147222,0.1666667,Septiembre
2,2,2019-09-01 00:27:06,2019-09-01 00:33:22,1,1.73,7.5,1.50,10.30,1,1,0.1044444,0.1456311,Septiembre
3,3,2019-09-01 00:43:23,2019-09-01 00:59:54,1,3.42,14.0,3.06,18.36,1,1,0.2752778,0.1666667,Septiembre
4,4,2019-09-01 00:28:55,2019-09-01 00:52:09,1,4.34,18.0,3.86,23.16,1,1,0.3872222,0.1666667,Septiembre
5,5,2019-09-01 00:13:37,2019-09-01 00:26:46,1,3.59,13.0,5.12,22.17,1,1,0.2191667,0.2309427,Septiembre
6,6,2019-09-01 00:35:43,2019-09-01 00:47:21,1,2.10,9.5,1.70,13.00,1,1,0.1938889,0.1307692,Septiembre


### ¡Funciona! Procedamos a hacer el análisis estadístico que nos interesa. 